In [11]:
import os
lines = {}
for dirname, dirpaths, files in os.walk('.'):
    if "history_codes" in dirname: continue
    for file in files:
        if file.endswith('.py'):
            filepath = os.path.join(dirname, file)
            with open(filepath) as f:
                lines = {
                    **lines,
                    **{filepath: len(f.readlines())}
                }
[print(line) for line in lines.items()]
print(f"Total lines of code: {sum(lines.values())}")

('./eval.py', 100)
('./meta.py', 160)
('./meta_bak.py', 337)
('./main.py', 217)
('./show_result.py', 257)
('./train_test_meta.py', 379)
('./best_results.py', 334)
('./train_test.py', 411)
('./models/LSTM_ONLY.py', 34)
('./models/MPNN_LSTM.py', 198)
('./models/others.py', 246)
('./models/dynst.py', 283)
('./utils/model_selector.py', 53)
('./utils/utils.py', 316)
('./utils/args.py', 207)
('./utils/logger.py', 89)
('./utils/test.py', 156)
('./utils/datetime.py', 141)
('./utils/datasets.py', 525)
('./visualization/vis.py', 208)
Total lines of code: 4651
